In [1]:
import warnings

# import install package/module
import pandas as pd
import numpy as np
import scipy as sp
from datetime import datetime
from dateutil.relativedelta import relativedelta
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from google.cloud import bigquery

pd.set_option('display.max_columns', None)

pd.set_option('display.float_format', lambda x: '%.3f' % x)

# configuration
warnings.simplefilter(action='ignore', category=FutureWarning)

with warnings.catch_warnings():
    warnings.simplefilter('ignore', category=UserWarning)
    client = bigquery.Client(project="inunaki")

%config InlineBackend.figure_format = "retina"

In [2]:
df_plan = pd.read_csv('csv/plan_data_202409_sue.csv', encoding='UTF-8')

df_plan['jan'] = df_plan['jan'].astype(str).str.zfill(20)
df_plan['store_cd'] = df_plan['store_cd'].astype(str)
df_plan['location'] = df_plan['location'].astype(str)

df_real = pd.read_csv('csv/real_data_202409_sue.csv')

df_real['jan'] = df_real['jan'].astype(str).str.zfill(20)
df_real['store_cd'] = df_real['store_cd'].astype(str)
df_real['location'] = df_real['location'].astype(str)

real_jan = df_real['jan'].tolist()
df_plan['real_flg'] = df_plan['jan'].apply(lambda x: 1 if x in real_jan else 0)

plan_jan = df_plan['jan'].tolist()
df_real['plan_flg'] = df_real['jan'].apply(lambda x: 1 if x in plan_jan else 0)

In [3]:
percent_store_cd = '473'

store = df_plan[(df_plan['store_cd']== percent_store_cd) & (df_plan['real_flg']==1)]['jan'].nunique()/ df_plan[df_plan['store_cd']== percent_store_cd]['jan'].nunique()
labels = ['whole','store']
values = [1-store, store ]
colors = ['#D6E6DF', '#D7637A']
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.5, marker=dict(colors=colors))])
fig.update_traces(hoverinfo='none', textinfo='none', showlegend=False)
fig.update_layout(
    annotations=[dict(text=f"{store:.2%}", x=0.5, y=0.5, font=dict(size=23, color='#D7637A'), showarrow=False)])
fig.show()
print('計画SKU数:', df_plan['jan'].nunique())
print('実棚SKU数:', df_real['jan'].nunique())
print('一致SKU数:', df_plan[df_plan['real_flg']==1]['jan'].nunique())

計画SKU数: 347
実棚SKU数: 561
一致SKU数: 285
